# Joining the Aggregated Data Together 
## Author: Dulan Wijeratne 1181873

In this notebook we will join the previously aggregated data together. 

We will start by starting a spark session and then reading in the data.

In [1]:
from pyspark.sql import SparkSession, functions as f

In [2]:
spark = (
    SparkSession.builder.appName("Preprocessing_Yellow")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config('spark.driver.memory', '3g')   
    .config('spark.executor.memory', '4g')  
    .config('spark.executor.instances', '2')  
    .config('spark.executor.cores', '2')
    .getOrCreate()
)

your 131072x1 screen size is bogus. expect trouble
23/10/02 22:21:53 WARN Utils: Your hostname, LAPTOP-RELH58H1 resolves to a loopback address: 127.0.1.1; using 172.19.22.4 instead (on interface eth0)
23/10/02 22:21:53 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/10/02 22:21:55 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
merchant_agg = spark.read.parquet("../../../data/insights/agg_insight_data/merchant_agg.parquet/")
consumers_agg = spark.read.parquet("../../../data/insights/agg_insight_data/consumers_agg.parquet/")
orders_agg = spark.read.parquet("../../../data/insights/agg_insight_data/orders_agg.parquet/")
postcode_agg = spark.read.parquet("../../../data/insights/agg_insight_data/postcode_agg.parquet/")
descriptions_agg = spark.read.parquet("../../../data/insights/agg_insight_data/descriptions_agg.parquet/")

Next we want to check for duplicate or unnecessary

In [4]:
print(merchant_agg.columns)

['merchant_abn', 'name', 'revenue_level', 'take_rate', 'average_merchant_fraud_probability']


In [5]:
print(consumers_agg.columns)

['merchant_abn', 'number_of_unique_consumers', 'average_consumer_fraud_probability', 'number_of_repeat_consumers', 'average_repeat_transactions_per_consumer']


In [6]:
print(orders_agg.columns)

['merchant_abn', 'number_of_orders', 'average_cost_of_order', 'average_spend_per_consumer', 'average_monthly_diff_consumers', 'consumer_diff_over_period', 'average_growth_consumers', 'merchant_revenue_rounded', 'first_recorded_transaction', 'last_recorded_transaction', 'transcation_period_months']


In [7]:
print(postcode_agg.columns)

['merchant_abn', 'name', 'number_of_postcodes', 'avg_total_weekly_personal_income', 'avg_total_weekly_fam_income', 'avg_median_age', 'avg_household_size', 'postcode_reach', 'avg_num_of_consumers_per_postcode']


In [8]:
postcode_agg = postcode_agg.drop("name")

In [9]:
print(descriptions_agg.columns)

['merchant_abn', 'merchant_description', 'segment']


In [10]:
descriptions_agg = descriptions_agg.drop("merchant_description")

### Joining the Data

In [11]:
joined_df = merchant_agg.join(consumers_agg, on = "merchant_abn", how = "inner").\
                join(orders_agg, on = "merchant_abn", how = "inner").\
                join(postcode_agg, on = "merchant_abn", how = "inner").\
                join(descriptions_agg, on = "merchant_abn", how = "inner")

In [12]:
joined_df = joined_df.fillna(0)

In [13]:
joined_df.show()

23/10/02 22:22:08 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+------------+--------------------+-------------+---------+----------------------------------+--------------------------+----------------------------------+--------------------------+----------------------------------------+----------------+---------------------+--------------------------+------------------------------+-------------------------+------------------------+------------------------+--------------------------+-------------------------+-------------------------+-------------------+--------------------------------+---------------------------+------------------+------------------+--------------------+---------------------------------+--------------------+
|merchant_abn|                name|revenue_level|take_rate|average_merchant_fraud_probability|number_of_unique_consumers|average_consumer_fraud_probability|number_of_repeat_consumers|average_repeat_transactions_per_consumer|number_of_orders|average_cost_of_order|average_spend_per_consumer|average_monthly_diff_consumers|consume

In [14]:
joined_df = joined_df.withColumn("bnpl_maximum_gain", (joined_df.take_rate/100) * joined_df.merchant_revenue_rounded)

### Saving Data

In [15]:
joined_df.write.mode("overwrite").parquet("../../../data/insights/joined.parquet")

In [16]:
# number of postcodes / total number of postcodes = reach

# number of consumers / number of repeat consumers = customer retainability

# average number of repeat consumer transaction per merchant.

# avg spend amount per consumer

# estimated consumer growth rate (group by month and calculate average increase / decrease)

# estimated revenue growth rate (group by month and calculate average increase / decrease)



In [17]:
spark.stop()